In [1]:
import numpy as np
import scanpy as sc
import anndata
import scvi
from scib_metrics.benchmark import Benchmarker
import torch
import scib_metrics
from lightning.pytorch.callbacks import ModelCheckpoint
import copy
%matplotlib inline
import sys
sys.path.append('..')
from functions import *
from SCVIModelCheckpoint import SCVIModelCheckpoint 

/local/data/scvi-tools/scvi/_settings.py:63: UserWarning: Since v1.0.0, scvi-tools no longer uses a random seed by default. Run `scvi.settings.seed = 0` to reproduce results from previous versions.
  self.seed = seed
/local/data/scvi-tools/scvi/_settings.py:70: UserWarning: Setting `dl_pin_memory_gpu_training` is deprecated in v1.0 and will be removed in v1.1. Please pass in `pin_memory` to the data loaders instead.
  self.dl_pin_memory_gpu_training = (
/local/data/anaconda3/envs/BT/lib/python3.11/site-packages/numba/np/ufunc/dufunc.py:84: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  dispatcher = jit(_target='npyufunc',
/local/data/anaconda3/envs/BT/lib/python3.11/site-pac

In [30]:
adata = anndata.read_h5ad(filename="../data/adataLung4.h5ad")

In [31]:
adata

AnnData object with n_obs × n_vars = 32472 × 4000
    obs: 'dataset', 'location', 'nGene', 'nUMI', 'patientGroup', 'percent.mito', 'protocol', 'sanger_type', 'size_factors', 'sampling_method', 'batch', 'cell_type', 'donor'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'highly_variable_nbatches', 'highly_variable_intersection'
    uns: 'hvg', 'pca'
    obsm: 'Scanorama', 'Unintegrated', 'X_pca'
    varm: 'PCs'
    layers: 'counts'

In [35]:
train_adata = adata[:int(32472*0.9)]
test_adata = adata[int(32472*0.9):]

In [4]:
ks = [2, 5, 10, 50, 100, 200, 500]

In [36]:
def hyperTraining2(adata, prior, hyperpar, ks, n_epochs_kl_warmup=300, max_epochs=400, beta = 5, early_stopping=False):
    adatac = adata.copy()
    keys = []
    vaes = []
    for k in ks:
        adataPrior, vae = trainModel(adata, prior, beta, n_epochs_kl_warmup=n_epochs_kl_warmup,max_epochs=max_epochs,early_stopping=early_stopping,save=f"models/Hyper_Exp/HyperExp_{prior}_{k}",prior_kwargs={hyperpar:k},log=True,logname=f"hyperTrainingMGk={k}")
        keys.append(f"scVI_k={k}")
        adatac.obsm[f"scVI_k={k}"] = vae.get_latent_representation()
        vaes.append(vae)
    return adatac, keys, vaes

In [37]:
adataMG, keysMG, vaes = hyperTraining2(train_adata, "mixofgaus", "k", ks)
bmMG = runBenchmark(adataMG, keysMG, nmi_ari_cluster_labels_leiden=True, nmi_ari_cluster_labels_kmeans = True, silhouette_label=True, silhouette_batch = True, kbet_per_label=True)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: lightning_logs/hyperTrainingMGk=2
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 400/400: 100%|██████████| 400/400 [13:07<00:00,  1.65s/it, v_num=0, train_loss_step=960, train_loss_epoch=883]    

`Trainer.fit` stopped: `max_epochs=400` reached.


Epoch 400/400: 100%|██████████| 400/400 [13:07<00:00,  1.97s/it, v_num=0, train_loss_step=960, train_loss_epoch=883]


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: lightning_logs/hyperTrainingMGk=5
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


...

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: lightning_logs/hyperTrainingMGk=500
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 400/400: 100%|██████████| 400/400 [12:03<00:00,  1.84s/it, v_num=0, train_loss_step=941, train_loss_epoch=880]    

`Trainer.fit` stopped: `max_epochs=400` reached.


Epoch 400/400: 100%|██████████| 400/400 [12:03<00:00,  1.81s/it, v_num=0, train_loss_step=941, train_loss_epoch=880]


Embeddings: 100%|██████████| 7/7 [02:51<00:00, 24.44s/it]


In [38]:
results = bmMG.get_results(min_max_scale=False)
results.to_csv("results/parameter_optimization_MG.csv")

In [39]:
lls = []
for i in range(7):
    lls.append(vaes[i].get_marginal_ll(test_adata))

INFO     Received view of anndata, making copy.                                                                    
INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             
INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             
INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             
INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             
INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             
INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             
INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             


In [44]:
results

,Leiden NMI,Leiden ARI,KMeans NMI,KMeans ARI,Silhouette label,Silhouette batch,KBET,Batch correction,Bio conservation,Total,loglik,k
Embedding,,,,,,,,,,,,
scVI_k=2,0.642188,0.449975,0.581017,0.383407,0.556002,0.83982,0.403353,0.621586,0.522518,0.562145,-940.664398,2
scVI_k=5,0.685009,0.519224,0.591168,0.417928,0.576314,0.845477,0.395776,0.620627,0.557929,0.583008,-935.720478,5
scVI_k=10,0.656548,0.526381,0.618108,0.448333,0.569116,0.805526,0.463142,0.634334,0.563697,0.591952,-924.170175,10
scVI_k=50,0.675619,0.49501,0.620439,0.449195,0.585932,0.798792,0.484644,0.641718,0.565239,0.595831,-917.926899,50
scVI_k=100,0.683498,0.536135,0.64186,0.505328,0.604994,0.795855,0.497109,0.646482,0.594363,0.61521,-918.417962,100
scVI_k=200,0.695762,0.568009,0.642996,0.506746,0.599106,0.801907,0.485407,0.643657,0.602524,0.618977,-918.390104,200
scVI_k=500,0.682361,0.537061,0.652402,0.463339,0.607388,0.802706,0.496004,0.649355,0.58851,0.612848,-925.819545,500


In [43]:
results["loglik"] = lls
results["k"] = ks

type: "['Metric Type'] not found in axis"

In [40]:
results

,Leiden NMI,Leiden ARI,KMeans NMI,KMeans ARI,Silhouette label,Silhouette batch,KBET,Batch correction,Bio conservation,Total
Embedding,,,,,,,,,,
scVI_k=2,0.642188,0.449975,0.581017,0.383407,0.556002,0.83982,0.403353,0.621586,0.522518,0.562145
scVI_k=5,0.685009,0.519224,0.591168,0.417928,0.576314,0.845477,0.395776,0.620627,0.557929,0.583008
scVI_k=10,0.656548,0.526381,0.618108,0.448333,0.569116,0.805526,0.463142,0.634334,0.563697,0.591952
scVI_k=50,0.675619,0.49501,0.620439,0.449195,0.585932,0.798792,0.484644,0.641718,0.565239,0.595831
scVI_k=100,0.683498,0.536135,0.64186,0.505328,0.604994,0.795855,0.497109,0.646482,0.594363,0.61521
scVI_k=200,0.695762,0.568009,0.642996,0.506746,0.599106,0.801907,0.485407,0.643657,0.602524,0.618977
scVI_k=500,0.682361,0.537061,0.652402,0.463339,0.607388,0.802706,0.496004,0.649355,0.58851,0.612848
Metric Type,Bio conservation,Bio conservation,Bio conservation,Bio conservation,Bio conservation,Batch correction,Batch correction,Aggregate score,Aggregate score,Aggregate score


In [50]:
plot = sns.relplot(data = results, x="loglik", y="Total")
for i in range(results.shape[0]):
    plt.annotate(results["k"][i],(results["loglik"][i],results["Total"][i]+0.001))
plot.set(title="Parameter Tuning")
plot.savefig("plots/prior_optimization")

AnnData object with n_obs × n_vars = 32472 × 4000
    obs: 'dataset', 'location', 'nGene', 'nUMI', 'patientGroup', 'percent.mito', 'protocol', 'sanger_type', 'size_factors', 'sampling_method', 'batch', 'cell_type', 'donor'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'highly_variable_nbatches', 'highly_variable_intersection'
    uns: 'hvg', 'pca'
    obsm: 'Scanorama', 'Unintegrated', 'X_pca', 'scVI_k=2', 'scVI_k=5', 'scVI_k=10', 'scVI_k=50', 'scVI_k=100', 'scVI_k=200', 'scVI_k=500'
    varm: 'PCs'
    layers: 'counts'